# Natural Capital Pipeline (Colab)
このノートブックは上から順に実行してください。

## Input 1: Google Drive をマウントして作業フォルダを準備

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

import os
PROJECT_DIR = "/content/drive/MyDrive/pj-TERM"
os.makedirs(PROJECT_DIR, exist_ok=True)
%cd /content/drive/MyDrive/pj-TERM


## Input 2: リポジトリを配置/更新

In [ ]:
import os

REPO_URL = "https://github.com/KotaroOmote/26s-TERM-01.git"
TARGET = "/content/drive/MyDrive/pj-TERM"

if not os.path.exists(os.path.join(TARGET, ".git")):
    !git clone {REPO_URL} {TARGET}
else:
    %cd {TARGET}
    !git pull --rebase

%cd /content/drive/MyDrive/pj-TERM
!ls -la


## Input 3: パイプライン実行 (run / from-cache)

In [ ]:
!python natural_capital_pipeline.py run --config config_example.json
!python natural_capital_pipeline.py from-cache --config config_example.json


## Input 4: インデックス・リターン・ボラティリティ・金融シグナルを可視化

In [ ]:
import json
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

base = Path("cache/fujisawa_demo/derived")
idx = pd.read_csv(base / "natural_capital_index.csv")
vol = pd.read_csv(base / "volatility_forecast.csv")
signals = json.loads((base / "finance_signals.json").read_text(encoding="utf-8"))

idx["week_start"] = pd.to_datetime(idx["week_start"])
idx = idx.sort_values("week_start").reset_index(drop=True)
idx["ret_vol_12w"] = idx["ecological_return"].rolling(12).std()

thr = signals["derivative"]["vol_threshold"]
triggered = int(signals["derivative"]["triggered"])
payout_ratio_pct = 100 * signals["derivative"]["payout_ratio"]
base_coupon_pct = 100 * signals["nature_bond"]["base_coupon"]
final_coupon_pct = 100 * signals["nature_bond"]["final_coupon"]

fig, axes = plt.subplots(2, 2, figsize=(14, 9), constrained_layout=True)

axes[0, 0].plot(idx["week_start"], idx["natural_capital_index"], lw=2)
axes[0, 0].set_title("Natural Capital Index")
axes[0, 0].set_xlabel("Week")
axes[0, 0].set_ylabel("Index")
axes[0, 0].grid(alpha=0.3)

ax2 = axes[0, 1]
ax2.plot(idx["week_start"], idx["ecological_return"], lw=1.5, label="Ecological Return")
ax2.set_title("Ecological Return")
ax2.set_xlabel("Week")
ax2.set_ylabel("Return")
ax2.grid(alpha=0.3)

ax2b = ax2.twinx()
ax2b.plot(idx["week_start"], idx["ret_vol_12w"], ls="--", lw=1.5, label="12w Volatility")
ax2b.set_ylabel("Volatility (std)")

l1, lb1 = ax2.get_legend_handles_labels()
l2, lb2 = ax2b.get_legend_handles_labels()
ax2.legend(l1 + l2, lb1 + lb2, loc="upper right")

axes[1, 0].plot(vol["horizon_week"], vol["garch_sigma2"], marker="o", label="GARCH")
axes[1, 0].plot(vol["horizon_week"], vol["transformer_sigma2"], marker="s", label="Transformer-like")
axes[1, 0].axhline(thr, color="red", ls="--", label=f"Threshold={thr:.2e}")
axes[1, 0].set_title("Volatility Forecast (Sigma^2)")
axes[1, 0].set_xlabel("Horizon Week")
axes[1, 0].set_ylabel("Variance")
axes[1, 0].grid(alpha=0.3)
axes[1, 0].legend()

labels = ["Payout Ratio(%)", "Base Coupon(%)", "Final Coupon(%)", "Triggered(0/1)"]
vals = [payout_ratio_pct, base_coupon_pct, final_coupon_pct, triggered]
axes[1, 1].bar(labels, vals)
axes[1, 1].set_title("Finance Signals Summary")
axes[1, 1].grid(axis="y", alpha=0.3)
axes[1, 1].tick_params(axis="x", rotation=20)

plt.show()


## Input 5: 週次特徴量の Z-score 可視化

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

feat = pd.read_csv(Path("cache/fujisawa_demo/derived/features_weekly.csv"))
feat["week_start"] = pd.to_datetime(feat["week_start"])

num_cols = [c for c in feat.columns if c != "week_start"]
z = feat[num_cols].copy()
z = (z - z.mean()) / z.std().replace(0, 1)

plt.figure(figsize=(14, 5))
for c in num_cols:
    plt.plot(feat["week_start"], z[c], label=c, alpha=0.9)
plt.title("Weekly Features (Z-score normalized)")
plt.xlabel("Week")
plt.ylabel("Z-score")
plt.grid(alpha=0.3)
plt.legend(ncol=3, fontsize=9)
plt.show()
